In [13]:
from sentence_transformers import SentenceTransformer
import chromadb

# Load embedding model
model = SentenceTransformer("sentence-transformers/msmarco-MiniLM-L6-cos-v5")

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="sql_mappings")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:


# Load JSON file
with open("extended_acl_to_databricks_dataset.json", "r") as f:
    mappings = json.load(f)

# Store embeddings in ChromaDB
for i, mapping in enumerate(mappings):
    acl_sql = mapping["input"]
    databricks_sql = mapping["output"]
    embedding = model.encode(acl_sql).tolist()

    collection.add(
        ids=[str(i)],  # Unique ID
        embeddings=[embedding],
        metadatas=[{"databricks_sql": databricks_sql}]
    )

print("✅ JSON data stored in ChromaDB successfully!")
print(embedding[0:7])


Insert of existing embedding ID: 0
Add of existing embedding ID: 0
Insert of existing embedding ID: 1
Add of existing embedding ID: 1
Insert of existing embedding ID: 2
Add of existing embedding ID: 2
Insert of existing embedding ID: 3
Add of existing embedding ID: 3
Insert of existing embedding ID: 4
Add of existing embedding ID: 4
Insert of existing embedding ID: 5
Add of existing embedding ID: 5
Insert of existing embedding ID: 6
Add of existing embedding ID: 6
Insert of existing embedding ID: 7
Add of existing embedding ID: 7
Insert of existing embedding ID: 8
Add of existing embedding ID: 8
Insert of existing embedding ID: 9
Add of existing embedding ID: 9
Insert of existing embedding ID: 10
Add of existing embedding ID: 10
Insert of existing embedding ID: 11
Add of existing embedding ID: 11
Insert of existing embedding ID: 12
Add of existing embedding ID: 12
Insert of existing embedding ID: 13
Add of existing embedding ID: 13
Insert of existing embedding ID: 14
Add of existing em

✅ JSON data stored in ChromaDB successfully!
[0.05647555738687515, 0.04955434054136276, -0.045201078057289124, -0.052149251103401184, -0.06306476891040802, 0.04201562702655792, 0.10238372534513474]


In [31]:
from sentence_transformers import SentenceTransformer
import chromadb
import json
import ollama

# Load the ChromaDB client and model
model = SentenceTransformer("sentence-transformers/msmarco-MiniLM-L6-cos-v5")
chroma_client = chromadb.PersistentClient(path="./chroma_db")  # Use your ChromaDB path
collection = chroma_client.get_or_create_collection(name="sql_mappings")

# Function to convert new ACL query to Databricks SQL
def convert_acl_to_databricks(acl_query):
    # Step 1: Generate the embedding for the new ACL query
    acl_embedding = model.encode(acl_query).tolist()

    # Step 2: Retrieve the closest match from ChromaDB
    search_results = collection.query(
        query_embeddings=[acl_embedding], n_results=5  # Increase n_results if needed
    )

    # Debugging output for search results
    print("🔹 Debugging search results:", search_results)

    # Step 3: Check if a match is found in ChromaDB
    if search_results["documents"]:
        # Get the best match from ChromaDB metadata (Databricks SQL)
        best_match = search_results["metadatas"][0][0].get('databricks_sql', 'No Databricks SQL found')
        print("🔹 Best match found in ChromaDB:", best_match)
        return best_match
    else:
        # Step 4: Use Ollama to generate Databricks SQL if no match is found
        prompt = f"Convert this ACL query to a valid Databricks SQL query:\n{acl_query}"
        try:
            print(f"🔹 Asking Ollama to convert: {acl_query}")
            response = ollama.chat(model="deepseek-r1", messages=[{"role": "user", "content": prompt}])
            generated_sql = response["message"]["content"].strip()

            # Check if Ollama generated a valid response
            if generated_sql:
                print("🔹 Generated SQL from Ollama:", generated_sql)
                return generated_sql
            else:
                print(f"🔹 No valid SQL generated for: {acl_query}")
                return f"Error: No valid SQL generated for: {acl_query}"
        except Exception as e:
            print(f"Error with Ollama for query: {acl_query}, error: {e}")
            return f"Error with Ollama for query: {acl_query}"

# Example conversion for a new ACL query
new_acl_query = "SELECT ProductID, Department, CustomerID, Region, Quantity FROM orders WHERE Quantity < 10;"

# Get the converted Databricks SQL query
converted_query = convert_acl_to_databricks(new_acl_query)
print("🔹 Converted Query:", converted_query)


🔹 Debugging search results: {'ids': [['410', '346', '113', '46', '215']], 'embeddings': None, 'documents': [[None, None, None, None, None]], 'uris': None, 'data': None, 'metadatas': [[{'databricks_sql': "SELECT Quantity, CustomerID FROM products WHERE Department = 'Sales';"}, {'databricks_sql': "SELECT Year, HireDate, Region, CustomerID, Department, ProductID FROM orders WHERE ProductID = (SELECT ProductID FROM transactions WHERE Region = 'North');"}, {'databricks_sql': 'SELECT ProductID FROM customers WHERE CustomerID = (SELECT CustomerID FROM customers WHERE Quantity < 10);'}, {'databricks_sql': 'SELECT Year, Price, Region, OrderID, Salary, CustomerID, HireDate, Quantity FROM departments WHERE ProductID = (SELECT ProductID FROM sales WHERE Year = 2023);'}, {'databricks_sql': "SELECT OrderID, Quantity, Price, HireDate, Department, CustomerID, ProductID, Salary, Year, Region FROM transactions WHERE OrderID = (SELECT OrderID FROM transactions WHERE Department = 'Sales');"}]], 'distances

In [41]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import T5Tokenizer, T5ForConditionalGeneration
# Assuming 'model' is the pre-trained sentence transformer (e.g., `msmarco-MiniLM-L6-cos-v5`)
# ChromaDB and collection are already initialized

# Initialize Ollama (DeepSeek)

def convert_acl_to_databricks(new_acl_query):
    # Step 1: Generate embedding for the new ACL query
    new_acl_embedding = model.encode(new_acl_query).tolist()

    # Step 2: Retrieve embeddings from ChromaDB
    search_results = collection.query(
        query_embeddings=[new_acl_embedding], n_results=5  # Retrieve top 5 matches
    )

    # Debugging output to show the search results
    print("🔹 Debugging search results:", search_results)

    # Step 3: Find the most similar match based on cosine similarity
    best_match = None
    best_distance = float('inf')

    # Iterate over the search results and find the best match based on distance
    for idx, result in enumerate(search_results["distances"]):
        # Ensure 'result' is a float and not a list
        distance = result[0] if isinstance(result, list) else result
        if distance < best_distance:
            best_match = search_results["metadatas"][idx][0].get('databricks_sql', 'No Databricks SQL found')
            best_distance = distance

    # If a good match is found in ChromaDB (threshold can be adjusted)
    if best_match and best_distance < 0.1:  # Adjust threshold to control similarity
        print("🔹 Best match found in ChromaDB:", best_match)
        return best_match
    else:
        # Step 4: If no good match is found, use DeepSeek (Ollama) to generate the Databricks SQL query
        print(f"🔹 No good match found in ChromaDB. Asking DeepSeek (Ollama) to convert: {new_acl_query}")
        
        # Pass the new ACL query to DeepSeek for Databricks SQL generation
        prompt = f"Convert this ACL query to a valid Databricks SQL query.\nACL Query: {new_acl_query}"
        
        try:
            response = ollama.chat(model = T5ForConditionalGeneration.from_pretrained("t5-small"), messages=[{"role": "user", "content": prompt}])
            generated_sql = response["message"]["content"].strip()

            print("🔹 Generated SQL from DeepSeek:", generated_sql)
            return generated_sql

        except Exception as e:
            print(f"Error with DeepSeek for query: {new_acl_query}, error: {e}")
            return f"Error with DeepSeek for query: {new_acl_query}"

# Example conversion for a new ACL query
new_acl_query = "SELECT ProductID, Department, CustomerID, Region, Quantity FROM orders WHERE Quantity < 10;"

# Get the converted Databricks SQL query
converted_query = convert_acl_to_databricks(new_acl_query)
print("🔹 Converted Query:", converted_query)


🔹 Debugging search results: {'ids': [['410', '346', '113', '46', '215']], 'embeddings': None, 'documents': [[None, None, None, None, None]], 'uris': None, 'data': None, 'metadatas': [[{'databricks_sql': "SELECT Quantity, CustomerID FROM products WHERE Department = 'Sales';"}, {'databricks_sql': "SELECT Year, HireDate, Region, CustomerID, Department, ProductID FROM orders WHERE ProductID = (SELECT ProductID FROM transactions WHERE Region = 'North');"}, {'databricks_sql': 'SELECT ProductID FROM customers WHERE CustomerID = (SELECT CustomerID FROM customers WHERE Quantity < 10);'}, {'databricks_sql': 'SELECT Year, Price, Region, OrderID, Salary, CustomerID, HireDate, Quantity FROM departments WHERE ProductID = (SELECT ProductID FROM sales WHERE Year = 2023);'}, {'databricks_sql': "SELECT OrderID, Quantity, Price, HireDate, Department, CustomerID, ProductID, Salary, Year, Region FROM transactions WHERE OrderID = (SELECT OrderID FROM transactions WHERE Department = 'Sales');"}]], 'distances

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the pre-trained T5 model and tokenizer
model_name = "t5-small"  # You can also use other T5 models like "t5-base" or "t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

def generate_sql_with_t5(prompt):
    # Encode the input prompt (ACL query)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Generate the SQL query using the T5 model
    outputs = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)

    # Decode the generated output to a string (SQL query)
    generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_sql

# Example usage
new_acl_query = "SELECT ProductID, Department, CustomerID, Region, Quantity FROM orders WHERE Quantity < 10;"
generated_sql = generate_sql_with_t5(new_acl_query)

# Print the generated SQL query
print("🔹 Generated SQL using T5:", generated_sql)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


🔹 Generated SQL using T5: SELECT ProductID, Department, CustomerID, Region, Quantity FROM orders WHERE Quantity  10;


In [44]:
pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 650.1 kB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.
